In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from signals import *
from frequencyestimator import *
from scipy.optimize import basinhopping, minimize
from tqdm.auto import tqdm

sns.set_style("whitegrid")
sns.despine(left=True, bottom=True)
sns.set_context("poster", font_scale = .45, rc={"grid.linewidth": 0.8})

/Users/farrokhlabib/Documents/github/csAE/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<Figure size 640x480 with 0 Axes>

In [2]:
P0 = lambda n, theta: np.cos((2*n+1)*theta)**2
P1 = lambda n, theta: np.sin((2*n+1)*theta)**2

def estimate_signal(depths, n_samples, theta):
        signals = np.zeros(len(depths), dtype = np.complex128)
        measurements = np.zeros(len(depths))
        for i,n in enumerate(depths):
            # Get the exact measuremnt probabilities
            p0 = P0(n, theta)
            p1 = P1(n, theta)

            p0x = P0x(n,theta)
            p1x = P1x(n,theta)
            
            p0_estimate = np.random.binomial(n_samples[i], p0)/n_samples[i]
            p1_estimate = 1 - p0_estimate

            p0x_estimate = np.random.binomial(n_samples[i], p0x)/n_samples[i]
            p1x_estimate = 1.0 - p0x_estimate
            measurements[i] = p0_estimate
            
            # Estimate theta
            theta_estimated = np.arctan2(p0x_estimate - p1x_estimate, p0_estimate - p1_estimate)

            # Compute f(n) - Eq. 3
            fi_estimate = np.exp(1.0j*theta_estimated)
            signals[i] = fi_estimate
         
        return signals, measurements

from scipy.stats import binom

def apply_correction(ula_signal, measurements, theta_est, theta):
    theta_est = np.abs(theta_est)
    p_exact = np.cos((2*ula_signal.depths+1)*(theta))**2
    p_neg = np.cos((2*ula_signal.depths+1)*(-theta))**2
    p_o2 = np.cos((2 * ula_signal.depths + 1) * (theta_est/2.0)) ** 2
    p_o4 = np.cos((2 * ula_signal.depths + 1) * (theta_est/4.0)) ** 2
    p_same = np.cos((2*ula_signal.depths+1)*(theta_est))**2
    p_s2 = np.cos((2 * ula_signal.depths + 1) * (np.pi/2-theta_est)) ** 2
    p_s4 = np.cos((2 * ula_signal.depths + 1) * (np.pi / 4 - theta_est)) ** 2
    p_s2_o2 = np.cos((2 * ula_signal.depths + 1) * (np.pi / 2 - theta_est/2)) ** 2
    p_s4_o2 = np.cos((2 * ula_signal.depths + 1) * (np.pi / 4 - theta_est/2)) ** 2

    l_exact = np.sum(
        np.log([1e-75 + binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk],
                                  p_exact[kk]) for kk in
                range(len(ula_signal.n_samples))]))
    l_neg = np.sum(
        np.log([1e-75 + binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk],
                                  p_neg[kk]) for kk in
                range(len(ula_signal.n_samples))]))
    l_o2 = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk]*measurements[kk], ula_signal.n_samples[kk], p_o2[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    l_o4 = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk], p_o4[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    l_same = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk], p_same[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    l_s2 = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk], p_s2[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    l_s4 = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk], p_s4[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    l_s2_o2 = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk], p_s2_o2[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    l_s4_o2 = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk], p_s4_o2[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    
    which_correction = np.argmax([l_same, l_s2, l_s4, l_o2, l_o4, l_s2_o2, l_s4_o2])
    if which_correction == 1:
        theta_est = np.pi/2.0 - theta_est
    elif which_correction == 2:
        theta_est = np.pi/4.0 - theta_est
    elif which_correction == 3:
        theta_est = theta_est/2
    elif which_correction == 4:
        theta_est = theta_est/4
    elif which_correction == 5:
        theta_est = np.pi / 2.0 - 0.5 * theta_est
    elif which_correction == 6:
        theta_est = np.pi / 4.0 - 0.5 * theta_est
    # elif which_correction == 7:
    #     theta_est = -theta_est

    # print(f'FINAL ANGLE FOUND: {theta_est, theta}')

    return np.abs(theta_est)

In [3]:
import numpy as np
import itertools
from typing import List

def generate_all_sign_variations(signs_array: np.ndarray) -> np.ndarray:
    """
    Generate all possible sign variations by changing signs at all possible pairs of positions.
    
    Parameters:
    -----------
    signs_array : numpy.ndarray
        Original array of signs (typically containing 1 or -1)
    
    Returns:
    --------
    numpy.ndarray
        Array of all possible sign variation arrays
    """
    # Get all possible unique pairs of positions
    n = len(signs_array)
    position_pairs = list(itertools.combinations(range(n), 2))
    
    # Will store all variations
    all_variations = []
    
    # Iterate through all possible position pairs
    for pos1, pos2 in position_pairs:
        # Generate variations for this pair of positions
        pair_variations = generate_pair_variations(signs_array, pos1, pos2)
        all_variations.extend(pair_variations)
    
    return all_variations

def generate_adjacent_sign_variations(signs_array: np.ndarray, size: int) -> np.ndarray:
    """
    Generate sign variations by sliding a two-position window across the array.
    
    Parameters:
    -----------
    signs_array : numpy.ndarray
        Original array of signs (typically containing 1 or -1)
    
    Returns:
    --------
    numpy.ndarray
        Array of all possible sign variation arrays
    """
    # Total length of the array
    n = len(signs_array)
    
    # Will store all variations
    all_variations = []
    
    # Iterate through adjacent position pairs 
    # (0,1), (1,2), (2,3), ... until the second-to-last pair
    for pos1 in range(1, n - size + 1):
        pos = [pos1 + i for i in range(size)]
        
        # Generate variations for this pair of adjacent positions
        pair_variations = generate_pair_variations(signs_array, pos)
 
        all_variations.extend(pair_variations)
    
    return all_variations

def generate_pair_variations(signs_array: np.ndarray, pos: List[int]) -> List[np.ndarray]:
    """
    Generate sign variations for a specific pair of positions.
    
    Parameters:
    -----------
    signs_array : numpy.ndarray
        Original array of signs
    pos1 : int
        First position to modify
    pos2 : int
        Second position to modify
    
    Returns:
    --------
    List[numpy.ndarray]
        List of sign variation arrays
    """
    # Validate input positions
    # if pos1 < 0 or pos2 < 0 or pos1 >= len(signs_array) or pos2 >= len(signs_array):
    #     raise ValueError("Positions must be within the array bounds")
    
    # Generate all possible sign combinations for the two positions
    sign_combinations = list(itertools.product([-1, 1], repeat=len(pos)))
    
    # Create variations
    variations = []
    for combo in sign_combinations:
        # Create a copy of the original array
        variation = signs_array.copy()
        
        # Modify the two specified positions
        for i in range(len(combo)):
            variation[pos[i]] *= combo[i]
        
        variations.append(tuple(variation))
    
    return variations


# Example array
signs = np.array([1]*4)

# Generate variations for all position pairs
all_variations = generate_adjacent_sign_variations(signs, 3)

# print("Original array:", signs)
# print(f"\nTotal variations: {len(all_variations)}")
# print(f'4*len(signs)^2: {4*len(signs)**2}')
# print(f'2^len(signs): {2**len(signs)}')

print("\nFirst few variations:")
for var in all_variations:  # Print first 10 variations
    print(var)


First few variations:
(1, -1, -1, -1)
(1, -1, -1, 1)
(1, -1, 1, -1)
(1, -1, 1, 1)
(1, 1, -1, -1)
(1, 1, -1, 1)
(1, 1, 1, -1)
(1, 1, 1, 1)


## Define the objective function to minimize

In [4]:
def objective_function(lp, cos_signal, abs_sin, ula_signal, esprit):   
    signal = cos_signal + 1.0j * lp * abs_sin
    R = ula_signal.get_cov_matrix_toeplitz(signal)
    _, _ = esprit.estimate_theta_toeplitz(R)
    eigs = np.abs(esprit.eigs)[:2]
    obj = eigs[1] - eigs[0]

    return obj

## What is the distribution of the signs? 

In [59]:
narray = [2,2,2,2,2,3]

def get_heavy_signs(narray, steps):

    thetas = np.linspace(np.arcsin(0.09), np.arcsin(0.91), steps)
    avals = np.sin(thetas)

    ula_signal = TwoqULASignal(M=narray, C=3)

    num_mc = 1000

    sign_distributions = {}

    for a,theta in zip(avals,thetas):
        # theta = np.arcsin(a)
        distr = {}
        for i in range(num_mc):
            signal, _ = estimate_signal(ula_signal.depths, ula_signal.n_samples, theta)
                # print(measurements)
            signs = tuple(np.sign(np.imag(signal)))
            # print(signs)

            distr[signs] = distr.get(signs, 0.0) + 1/num_mc
        sign_distributions[str(a)] = distr

    def get_signs(sign_distribution):
        # Sort the dictionary by values in descending order
        sorted_data = dict(sorted(sign_distribution.items(), key=lambda item: item[1], reverse=True))

        # Create a new dictionary with cumulative sum close to 0.68
        cumulative_dict = {}
        cumulative_sum = 0.0

        for key, value in sorted_data.items():
            cumulative_dict[key] = value
            if cumulative_sum + value > 0.9:
                break
            cumulative_sum += value

        # Output the result
        return cumulative_dict
    
    ret_dict = {}
    for a in avals:
        distr = get_signs(sign_distributions[str(a)])
        ret_dict[str(a)] = distr
 
    # Normalize the values
    normalized_data = {}
    for key, sub_dict in ret_dict.items():
        total_sum = sum(sub_dict.values())
        normalized_data[key] = {k: v / total_sum for k, v in sub_dict.items()}

    return normalized_data

normalized_data = get_heavy_signs(narray, 10)

# normalized_data

In [60]:
import ast
# Taking a random sample based on the normalized probabilities
def sample_from_normalized(data, sample_size=1):
    keys = list(map(str, list(data.keys())))
    probabilities = list(data.values())
    sampled_keys = np.random.choice(a=keys, size=sample_size, p=probabilities)
    return [ast.literal_eval(t) for t in sampled_keys]
avals = list(normalized_data.keys())
# Get one random sample
sampled = sample_from_normalized(normalized_data[avals[0]], sample_size=3)
# print("Sampled keys:",sampled)

## Use the distribution of signs so we don't have to try all possible signs

In [215]:
np.random.seed(42)

a = 0.1
print(a)
theta = np.arcsin(a)

narray = [2,2,2,2,2,2,2,2]
heavy_signs = get_heavy_signs(narray, len(narray)**2)

ula_signal = TwoqULASignal(M=narray, C=5)
esprit = ESPIRIT()

def sample_signs(heavy_signs, sample_size=3):
    """
    Sample a set of sign variations from a learned sign distribution.

    Args:
        heavy_signs (dict): A dictionary where the keys are strings representing float values,
                           and the values are dictionaries with keys as sign vectors and values
                           as their corresponding probabilities.
        sample_size (int, optional): The number of sign variations to sample. Defaults to 3.

    Returns:
        dict: A dictionary where the keys are the same as the keys in `heavy_signs`,
              and the values are lists of sampled sign variations.
    """
    avals = list(heavy_signs.keys())
    signs_to_try = {}
    for a in avals:
        signs_to_try[a] = list(set(sample_from_normalized(heavy_signs[a], sample_size=sample_size)))

    return signs_to_try

def avals_to_usef(a0, avals, L=3):
    """
    Find the `L` values in `avals` that are closest to the given `a0` value.

    Args:
        a0 (float): The reference value to find the closest `L` values for.
        avals (list): A list of float values representing the keys in `heavy_signs`.
        L (int, optional): The number of closest values to return. Defaults to 3.

    Returns:
        list: A list of string representations of the `L` closest values to `a0` in `avals`.
    """
    avals = list(map(float, avals))

    left = 0
    right = len(avals)
    
    while left < right:
        mid = (left + right) // 2
        
        if avals[mid] <= a0:
            left = mid + 1
        else:
            right = mid

    avals_to_use = list(map(str, avals[left-L: left+L]))
    return avals_to_use

def all_signs_to_try(avals_to_use, signs_to_try, adjacency=2):
    """
    Generate all possible sign variations within a Hamming distance of 2 from the given `avals_to_use`.

    Args:
        avals_to_use (list): A list of string representations of float values.
        signs_to_try (dict): A dictionary where the keys are string representations of float values,
                            and the values are lists of sign variations.
        adjacency (int, optional): The maximum Hamming distance to consider for the sign variations.
                                  Defaults to 2.

    Returns:
        list: A list of all possible sign variations within the specified Hamming distance.
    """
    all_signs = []
    for a in avals_to_use:
        for x in signs_to_try[a]:
            hamming_distance_two_signs = generate_adjacent_sign_variations(np.array(x), adjacency)
            all_signs.extend(hamming_distance_two_signs)
    all_signs = list(set(all_signs))
    
    return all_signs

def minimize_obj(all_signs, cos_signal, abs_sin, ula_signal, esprit, disp):
    """
    Find the sign variation that minimizes the objective function.

    Args:
        all_signs (list): A list of all possible sign variations to consider.
        cos_signal (numpy.ndarray): The real part of the estimated signal.
        abs_sin (numpy.ndarray): The absolute value of the imaginary part of the estimated signal.
        ula_signal (ULASignal): An instance of the ULASignal class, containing information about the signal.
        esprit (ESPRIT): An instance of the ESPRIT class, used for estimating the signal parameters.
        disp (bool): If True, prints the current objective and the current best sign variation.

    Returns:
        numpy.ndarray: The sign variation that minimizes the objective function.
    """
    obj = 1.0
    for x in all_signs:
        curr_obj = objective_function(np.array(x), cos_signal, abs_sin, ula_signal, esprit)
        if curr_obj < obj:
            if disp:
                print(f'current objective: {curr_obj}')
                print(f'current best signs: {x}')
            obj = curr_obj
            x_star = np.array(x)

    return x_star

def csae_with_local_minimization(theta, ula_signal, esprit, heavy_signs, sample=False, correction=False, optimize=False, disp=False):
    """
    Perform CSAE (Compressive Sensing Angle Estimation) with local minimization.

    Args:
        theta (float): The true angle of the signal.
        ula_signal (ULASignal): An instance of the ULASignal class, containing information about the signal.
        esprit (ESPRIT): An instance of the ESPRIT class, used for estimating the signal parameters.
        heavy_signs (dict): A dictionary where the keys are strings representing float values,
                           and the values are dictionaries with keys as sign vectors and values
                           as their corresponding probabilities.
        sample (bool, optional): If True, samples sign variations from the learned sign distribution.
                                If False, uses the learned sign distribution directly. Defaults to False.
        correction (bool, optional): If True, applies a correction to the estimated angle. Defaults to False.
        optimize (bool, optional): If True, uses a sliding window approach to find the best sign variation.
                                  If False, uses the learned sign distribution directly. Defaults to False.
        disp (bool, optional): If True, prints additional information during the process. Defaults to False.

    Returns:
        dict: A dictionary containing the estimated angles, errors, number of queries, maximum depth,
              and other relevant information.
    """

    depths = ula_signal.depths
    n_samples = ula_signal.n_samples
    csignal, measurements = estimate_signal(depths, n_samples, theta)

    cos_signal = np.real(csignal)

    correct_signs = np.sign(np.imag(csignal))
    if disp:
        print(f'correct signs: {correct_signs}')
    abs_sin = np.abs(np.imag(csignal))

    obj = 1.0
    if sample:
        # step 1: sample signs from learned sign distribution
        signs_to_try = sample_signs(heavy_signs=heavy_signs, sample_size=3)
        avals = heavy_signs.keys()

        if optimize:
            # step 2: using rough estimate where the amplitude is, use the a-values around that estimate
            a0 = np.sqrt(0.5 - 0.5 * cos_signal[0])
            avals_to_use = avals_to_usef(a0, avals, L=3)

            if disp:
                print(f'rough estimate a: {a0}')
                print(f'avals to use: {avals_to_use}')

            # step 3: now vary the signs in a sliding window of size "adjacency"
            all_signs = all_signs_to_try(avals_to_use, signs_to_try, adjacency=2)

            if disp:
                print(f'number of signs Hamming distance two: {len(all_signs)}')

            # step 4: try all the signs pick the ones that minimize the objective function
            x_star = minimize_obj(all_signs, cos_signal, abs_sin, ula_signal, esprit, disp)

            # step 5 (optional): do one more sweep
            hamming_distance_one_signs = list(set(generate_adjacent_sign_variations(x_star, 1)))
            x_star = minimize_obj(hamming_distance_one_signs, cos_signal, abs_sin, ula_signal, esprit, disp)
        
        else:
            # here we don't vary the signs using a sliding window, but directly use the learned sign distribution (poor performance)
            all_signs = []
            for a in avals:
                all_signs.extend(heavy_signs[a])
            all_signs = list(set(all_signs))
            x_star = minimize_obj(all_signs, cos_signal, abs_sin, ula_signal, esprit, disp)

    else:
        avals = list(heavy_signs.keys())
        signs_to_try = {}
        for a in avals:
            signs_to_try[a] = heavy_signs[a]

        if optimize:
            # step 2: using rough estimate where the amplitude is, use the a-values around that estimate
            a0 = np.sqrt(0.5 - 0.5 * cos_signal[0])
            avals_to_use = avals_to_usef(a0, avals, L=3)

            if disp:
                print(f'rough estimate a: {a0}')
                print(f'avals to use: {avals_to_use}')

            # step 3: now vary the signs in a sliding window of size "adjacency"
            all_signs = all_signs_to_try(avals_to_use, signs_to_try, adjacency=2)

            if disp:
                print(f'number of signs Hamming distance two: {len(all_signs)}')

            # step 4: try all the signs pick the ones that minimize the objective function
            x_star = minimize_obj(all_signs, cos_signal, abs_sin, ula_signal, esprit, disp)

            # step 5 (optional): do one more sweep
            hamming_distance_one_signs = list(set(generate_adjacent_sign_variations(x_star, 1)))
            x_star = minimize_obj(hamming_distance_one_signs, cos_signal, abs_sin, ula_signal, esprit, disp)
        
        else:
            # here we don't vary the signs using a sliding window, but directly use the learned sign distribution (poor performance)
            all_signs = []
            for a in avals:
                all_signs.extend(heavy_signs[a])
            all_signs = list(set(all_signs))
            x_star = minimize_obj(all_signs, cos_signal, abs_sin, ula_signal, esprit, disp)


    # Optimization is done.

    if disp:
        print(x_star)
        
    signal = cos_signal + 1.0j * x_star * abs_sin
    R = ula_signal.get_cov_matrix_toeplitz(signal)
    theta_est, _ = esprit.estimate_theta_toeplitz(R)
    theta_est = np.abs(theta_est)
    eigs = np.abs(esprit.eigs)[:2]
    basin_obj = eigs[1] - eigs[0]
    if correction:
        theta_est = apply_correction(ula_signal, measurements, theta_est, theta) #apply correction
    if disp:
        print(f'basin obj: {basin_obj}')

    cR = ula_signal.get_cov_matrix_toeplitz(csignal)
    theta_est1, _ = esprit.estimate_theta_toeplitz(cR)
    eigs = np.abs(esprit.eigs)[:2]
    true_obj = eigs[1] - eigs[0]
    if disp:
        print(f'true obj: {true_obj}')

    # compute queries required
    num_queries = np.sum(np.array(ula_signal.depths)*np.array(ula_signal.n_samples)) + ula_signal.n_samples[0]
    max_single_query = np.max(ula_signal.depths)

    ret_dict = {'theta_est': theta_est, 'theta_est1': theta_est1, 
                'error': np.abs(np.sin(theta)-np.sin(theta_est)), 
                'error1': np.abs(np.sin(theta)-np.sin(theta_est1)), 
                'queries': num_queries, 'depth': max_single_query,
                'true_obj': true_obj, 'basin_obj': basin_obj,
                'x_star': x_star}
    
    return ret_dict

csae_with_local_minimization(theta, ula_signal, esprit, heavy_signs, sample=True, correction=True, optimize=True, disp=True)

0.1
correct signs: [ 1.  1.  1.  1. -1.  1.  1.  1.  1.]
rough estimate a: 0.18910752115495127
avals to use: ['0.13981296510949756', '0.15634532387578837', '0.1728339924311597', '0.18927436306884374', '0.20566184157877418', '0.22199184853142423']
number of signs Hamming distance two: 131
current objective: -111.34188994413876
current best signs: (1.0, -1.0, -1.0, 1.0, -1.0, 1.0, -1.0, -1.0, 1.0)
current objective: -308.8108115642312
current best signs: (1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, -1.0, -1.0)
current objective: -338.48863204266365
current best signs: (1.0, 1.0, 1.0, 1.0, -1.0, 1.0, -1.0, -1.0, -1.0)
current objective: -397.4516598486228
current best signs: (1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, 1.0, 1.0)
current objective: -7.1407537803791
current best signs: (1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0, 1.0, 1.0)
current objective: -14.37126797725773
current best signs: (1.0, 1.0, -1.0, 1.0, -1.0, 1.0, 1.0, 1.0, 1.0)
current objective: -119.19130475889435
current best signs: (1.0, 1.0, 

{'theta_est': 0.10012026433053868,
 'theta_est1': 0.10012026433053861,
 'error': 4.692056561071689e-05,
 'error1': 4.692056561078628e-05,
 'queries': 2600,
 'depth': 128,
 'true_obj': -397.45165984862274,
 'basin_obj': -397.45165984862274,
 'x_star': array([ 1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,  1.])}

## Let's do some statistics now

In [44]:
np.random.seed(42)

avals = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
# avals = [np.random.uniform(0.1, 0.9)]
# avals = [0.5, 0.6, 0.7, 0.8, 0.9]
# avals = [0.1, 0.2, 0.3]
avals = [np.random.uniform(0.1, 0.9) for _ in range(15)]
# narray = [2]*5 + [3]
narray = [2]*6

print('learning the distribution of signs...')
heavy_signs = get_heavy_signs(narray, int(6*len(narray)))
num_queries = np.zeros(len(avals), dtype=int)
max_single_query = np.zeros(len(avals), dtype=int)

num_mc=100
thetas = np.zeros((len(avals), num_mc))
errors = np.zeros((len(avals), num_mc))
thetas1 = np.zeros((len(avals), num_mc))
errors1 = np.zeros((len(avals), num_mc))

basin_obj = np.zeros((len(avals), num_mc))
true_obj = np.zeros((len(avals), num_mc))

ula_signal = TwoqULASignal(M=narray, C=5)
esprit = ESPIRIT()

for j,a in enumerate(avals):
    theta = np.arcsin(a)
    print(f'theta = {theta}')
    for i in tqdm(range(num_mc)):
        res = csae_with_local_minimization(theta, ula_signal, esprit, heavy_signs, sample=True, correction=True, optimize=True, disp=False)
        thetas[j][i] = res['theta_est']
        errors[j][i] = res['error']

        thetas1[j][i] = res['theta_est1']
        errors1[j][i] = res['error1']

        basin_obj[j][i] = res['basin_obj']
        true_obj[j][i] = res['true_obj']
    num_queries[j] = res['queries']
    max_single_query[j] = res['depth']

    print(f'constant factors query and depth: {np.percentile(errors[j], 95) * num_queries[j]}, {np.percentile(errors[j], 95) * max_single_query[j]}')

learning the distribution of signs...
theta = 0.41111546403370536


100%|██████████| 100/100 [00:09<00:00, 10.59it/s]


constant factors query and depth: 3.72703095250019, 0.178007448477621
theta = 1.0363905664439828


100%|██████████| 100/100 [00:11<00:00,  8.65it/s]


constant factors query and depth: 2.8161590689618894, 0.13450311971161263
theta = 0.7554209219922302


...

constant factors query and depth: 3.3371018725582364, 0.1593839700326322
theta = 0.2732593578259982


100%|██████████| 100/100 [00:08<00:00, 11.24it/s]


constant factors query and depth: 12.512037103448261, 0.5975898318064841
theta = 0.24799415401854524


100%|██████████| 100/100 [00:08<00:00, 11.36it/s]

constant factors query and depth: 8.297542340867073, 0.39630052971305424


In [45]:
for i in range(len(avals)):
    print(np.sum(np.where(np.array(basin_obj[i])/np.array(true_obj[i]) >= 0.99, 1, 0))/len(basin_obj[i]))

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0


In [54]:
i=6
np.array(basin_obj[i])/np.array(true_obj[i]), np.sum(np.where(np.array(basin_obj[i])/np.array(true_obj[i]) >= 0.99, 1, 0))/len(basin_obj[i])

(array([1.02497076, 1.        , 1.04609651, 1.        , 1.        ,
        1.16423963, 1.11683879, 1.        , 1.21549878, 1.00545266,
        1.        , 1.00587115, 1.        , 1.        , 1.01386831,
        1.        , 1.        , 1.        , 1.15791901, 1.09110154,
        1.        , 1.        , 1.06740551, 1.01965389, 1.00646104,
        1.28687647, 1.        , 1.52416662, 1.04571905, 1.0610772 ,
        1.        , 1.11935286, 1.        , 1.0100827 , 1.18268422,
        1.02999208, 1.15919422, 1.02643683, 1.        , 1.        ,
        1.        , 1.02617146, 1.        , 1.2230548 , 1.11724276,
        1.        , 1.00443219, 1.02663556, 1.        , 1.0493558 ,
        1.00366068, 1.07676022, 1.07986259, 1.07108175, 1.0474498 ,
        1.01119834, 1.00893675, 1.14512875, 1.10661684, 1.        ,
        1.01951679, 1.14077254, 1.        , 1.08806654, 1.1123385 ,
        1.02276714, 1.        , 1.        , 1.        , 1.19599272,
        1.        , 1.19277811, 1.        , 1.  

In [55]:
i=6
np.sin(thetas[i]), avals[i]

(array([0.14094327, 0.14210457, 0.24457584, 0.14602306, 0.14536724,
        0.14308264, 0.146503  , 0.14655331, 0.14266459, 0.14333197,
        0.14568156, 0.2404924 , 0.14866643, 0.1455585 , 0.24222038,
        0.14951485, 0.14763524, 0.14828353, 0.14691832, 0.14739484,
        0.14335949, 0.14508579, 0.24335739, 0.14311765, 0.14269354,
        0.2390811 , 0.14848022, 0.14848611, 0.14319923, 0.24153758,
        0.14173488, 0.13850548, 0.14380408, 0.14636911, 0.14619498,
        0.12697084, 0.14815267, 0.1362355 , 0.15107397, 0.14654374,
        0.14313246, 0.13757741, 0.14456808, 0.1392772 , 0.14617979,
        0.15083628, 0.1427158 , 0.1240662 , 0.14541335, 0.14267239,
        0.1225064 , 0.14628344, 0.12338441, 0.14737236, 0.13927985,
        0.14324154, 0.14549842, 0.14776629, 0.14466761, 0.1488627 ,
        0.14559947, 0.14314409, 0.1463547 , 0.14644242, 0.24336449,
        0.24143969, 0.14057451, 0.14640159, 0.14919144, 0.1498014 ,
        0.14791762, 0.12317957, 0.14626426, 0.14

In [56]:
i=6
thetas[i]/np.pi, np.arcsin(avals[i])/np.pi

(array([0.04501352, 0.04538693, 0.07864871, 0.04664737, 0.04643636,
        0.04570147, 0.0468018 , 0.04681799, 0.04556702, 0.04578166,
        0.04653749, 0.07730891, 0.04749807, 0.0464979 , 0.0778757 ,
        0.04777118, 0.04716617, 0.04737482, 0.04693545, 0.0470888 ,
        0.04579051, 0.04634582, 0.07824878, 0.04571273, 0.04557633,
        0.07684618, 0.04743813, 0.04744003, 0.04573897, 0.07765171,
        0.04526805, 0.04422985, 0.04593351, 0.04675872, 0.04670269,
        0.04052547, 0.0473327 , 0.04350038, 0.04827317, 0.04681491,
        0.0457175 , 0.04393159, 0.04617927, 0.04447791, 0.0466978 ,
        0.04819663, 0.04558349, 0.03959352, 0.0464512 , 0.04556953,
        0.0390932 , 0.04673115, 0.03937482, 0.04708157, 0.04447876,
        0.04575258, 0.04647857, 0.04720834, 0.04621129, 0.04756124,
        0.04651108, 0.04572123, 0.04675408, 0.04678231, 0.07825112,
        0.0776196 , 0.04489496, 0.04676917, 0.04766707, 0.04786343,
        0.04725705, 0.03930911, 0.04672498, 0.04

In [49]:
# np.random.seed(1)

# csignal, measurements = estimate_signal(ula_signal.depths, ula_signal.n_samples, np.arcsin(avals[i]))

# plt.plot(measurements, 'r', label='measurements')
# plt.plot(np.cos((2*ula_signal.depths + 1)*theta2)**2, 'b', label='theta2')
# plt.plot(np.cos((2*ula_signal.depths + 1)*theta1)**2, 'k', label='theta1')
# plt.legend()

In [57]:
perc = 68
avg = 0.0
for i in range(len(avals)):
    avg += np.percentile(errors[i], perc) * num_queries[i]/len(avals)
    print(f'constant factors query and depth ({perc}%) for {avals[i]}: {np.percentile(errors[i], perc) * num_queries[i]}, {np.percentile(errors[i], perc) * max_single_query[i]}')
avg

constant factors query and depth (68%) for 0.39963209507789: 2.2796823127361607, 0.10888034926501065
constant factors query and depth (68%) for 0.8605714451279329: 0.7321750165723332, 0.03496955303032039
constant factors query and depth (68%) for 0.685595153449124: 1.2038127001567607, 0.05749553194778558
constant factors query and depth (68%) for 0.5789267873576293: 1.6251469867587645, 0.07761896056161263
constant factors query and depth (68%) for 0.22481491235394924: 2.5553260585007473, 0.12204542368958794
constant factors query and depth (68%) for 0.22479561626896213: 2.028211079451353, 0.09686978289916909
constant factors query and depth (68%) for 0.14646688973455957: 3.5073533240260346, 0.16751538264004942
constant factors query and depth (68%) for 0.7929409166199481: 1.2533817969522738, 0.05986301119772054
constant factors query and depth (68%) for 0.5808920093945671: 1.6365906952796971, 0.07816552574470195
constant factors query and depth (68%) for 0.6664580622368363: 1.202730372

1.7749252175495722

In [58]:
perc = 68
avg = 0.0
for j in range(len(avals)):
    avg += 2*np.percentile(errors1[j], perc) * num_queries[j]/len(avals)
    print(f'constant factors query and depth with known signs for {avals[j]}: {2*np.percentile(errors1[j], perc) * num_queries[j]}, {2*np.percentile(errors1[j], perc) * max_single_query[j]}')
avg

constant factors query and depth with known signs for 0.39963209507789: 4.314108491844801, 0.20604697274482633
constant factors query and depth with known signs for 0.8605714451279329: 1.9583743787100234, 0.09353429868465783
constant factors query and depth with known signs for 0.685595153449124: 2.378280610609581, 0.1135895217007561
constant factors query and depth with known signs for 0.5789267873576293: 3.2963784252355137, 0.1574389695634872
constant factors query and depth with known signs for 0.22481491235394924: 4.486854440469963, 0.2142975255149833
constant factors query and depth with known signs for 0.22479561626896213: 3.869096446437671, 0.18479266609851563
constant factors query and depth with known signs for 0.14646688973455957: 4.175131427288325, 0.1994092621988454
constant factors query and depth with known signs for 0.7929409166199481: 2.1376129807041218, 0.10209494833213716
constant factors query and depth with known signs for 0.5808920093945671: 3.1774890203211665, 0.1

3.1067720837130697